## 데이터 준비

- 필수 라이브러리 import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
import seaborn as sns

from statsmodels.formula.api import ols, glm

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

- 대기질 데이터 로드

In [ ]:
air_data = pd.read_csv("2021년_전국_대기질_지수.csv")

- 질환 데이터 로드

In [ ]:
disease_data = pd.read_csv("2021년_환경성질환_이용정보.csv")

- 범주형 -> 수치형 변환법 표시

In [ ]:
disease_code = {"비염":0, "아토피":1, "천식":2}
gender_code = {"남자":0, "여자":1}

- 행정구역 축소 함수

In [ ]:
def parser(word):
    if '북' in word:
        return word[0]+'북'+' '
    if '남' in word:
        return word[0]+'남'+' '
    else:
        return word[:2]+' '

### 공통 전처리

- 대기질 데이터 : 월별 평균으로 변환, 병합 위한 월 열 추가

In [ ]:
air_data["측정일시"] = air_data["측정일시"].apply(lambda x : x[:-3])
air_data = air_data.groupby(["지역", "측정일시"]).mean().reset_index()
air_data["월"] =  air_data["측정일시"].apply(lambda x : int(x[-2:]))

- 질환 데이터 : 시도명 지역으로 변환, 요양개시연 삭제, 성별/질병코드 부여

In [ ]:
disease_data["시도명"] = disease_data["시도명"].apply(lambda x : parser(x))
disease_data["지역"] = disease_data["시도명"]+disease_data["시군구명"]

disease_data.drop(columns=["요양개시연", "시도명", "시군구명"], inplace=True)
disease_data.rename(columns={"요양개시월":"월"}, inplace=True)

disease_data["질환"] = disease_data["질환"].apply(lambda x : disease_code[x])
disease_data["성별"] = disease_data["성별"].apply(lambda x : gender_code[x])

## 1번 풀이

- 질환 데이터 전처리

In [ ]:
q1_disease = disease_data.copy()
q1_disease.drop(columns = ["성별", "연령군"], inplace = True)
q1_disease = q1_disease.groupby(["지역", "월", "질환"]).sum().reset_index()

- 문제에 맞게 결합

In [ ]:
q1_data = pd.merge(q1_disease, air_data, on=["지역", "월"], how='left')
q1_data

- ols 이용 회귀

In [ ]:
Rformula = "진료합계 ~ SO2 + CO + O3 + NO2 + PM10 + PM25 + 질환"
regression_result = ols(Rformula, data = q1_data).fit()

## 1번 정답

- 결과 표시

In [ ]:
regression_result.summary()

## 2번 풀이

- 대기질 데이터 전처리

In [ ]:
q2_air = air_data.copy()
q2_air.drop(columns=['SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25'], inplace=True)

- 질환 데이터 전처리

In [ ]:
q2_disease = disease_data.copy()
q2_disease.drop(columns = ["연령군", "질환"], inplace = True)
q2_disease = q2_disease.groupby(["지역", "월", "성별"]).sum().reset_index()

- 문제에 맞게 결합, 전처리

In [ ]:
q2_data = pd.merge(q2_disease, q2_air, on = ["지역", "월"])
q2_data = q2_data[["지역", "측정일시", "성별", "진료합계"]]
q2_data["진료비율"] = None
q2_data

- 데이터가 남 여로 반복됨을 확인 정확한 비율 구하기 위해 두 행씩 계산

In [ ]:
for i in range(0, len(q2_data.index), 2):
    month_sum = q2_data.iloc[i,-2] + q2_data.iloc[i+1,-2]
    q2_data.iloc[i,-1] = q2_data.iloc[i,-2] / month_sum
    q2_data.iloc[i+1,-1] = q2_data.iloc[i+1,-2] / month_sum
    
q2_data.drop(columns=["진료합계"], inplace=True)
q2_data

- 시각화에 쓸 부분 추출

In [ ]:
male_ratio = q2_data.loc[q2_data['성별']==0, '진료비율']
female_ratio = q2_data.loc[q2_data['성별']==1, '진료비율']

## 2번 정답

- 히스토그램으로 시각화

In [ ]:
sns.histplot(male_ratio, kde=True, color='blue', label='남자', stat='density')
sns.histplot(female_ratio, kde=True, color = 'magenta', label='여자', stat='density')

plt.title('성별에 따른 진료 비율')
plt.legend()
plt.show()

## 3번 풀이

- 질환 데이터 전처리

In [ ]:
q3_disease = disease_data.copy()
minor = ['0-5', '6-11', '12-17']

q3_disease = q3_disease.loc[q3_disease["연령군"].apply(lambda x : x in minor), :].reset_index(drop=True)
q3_disease.drop(columns=["연령군"], inplace=True)

q3_disease = q3_disease.groupby(["지역", "월", "질환", "성별"]).sum().reset_index()

- 문제에 맞게 결합

In [ ]:
q3_data = pd.merge(q3_disease, air_data, on=["지역", "월"], how='left')
q3_data

- 피처와 라벨 분리, 피처는 정규분포 형태로 변환

In [ ]:
y = q3_data['성별']
x = q3_data[['SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25', '질환', '진료합계']]

scaler = StandardScaler()
x = scaler.fit_transform(x)

- 훈련-테스트 데이터 분리, 로지스틱 회귀 모델 생성, 학습, 적용

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

model = LogisticRegression()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

- 모델 평가 지수 계산

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

## 3번 정답

In [ ]:
print('정확도: {0:.3f}, 정밀도: {1:.3f}, 재현율: {2:.3f}, F1: {3:.3f}'.format(accuracy, precision, recall, f1))
print('ROC_AUC: {0:.3f}'.format(roc_auc))